In [ ]:
# Code from https://github.com/sv07i/Face-Detection/blob/main/Face%20Detection.py based on:
# https://www.computervision.zone/courses/468-face-landmarks/ and 
# https://www.youtube.com/watch?v=V9bzew8A1tc
import cv2
import mediapipe as mp
import time
import numpy as np
from mesh_points import MESH_ANNOTATIONS

pTime = 0
# cap = cv2.VideoCapture('/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still/IR_20s/Frame%05d.pgm')
cap = cv2.VideoCapture('/home/desafio01/Videos/Webcam/2021-05-14-190903.webm')
# cap = cv2.VideoCapture(0)

mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawspec = mpDraw.DrawingSpec(thickness=1, circle_radius=1)
blank = None
landmarks_ids = []
# landmarks_ids += MESH_ANNOTATIONS['bottom_face']
# landmarks_ids += MESH_ANNOTATIONS['leftEye']
# landmarks_ids += MESH_ANNOTATIONS['rightEye']
# landmarks_ids += MESH_ANNOTATIONS['mouth']
# landmarks_ids += MESH_ANNOTATIONS['nose']
landmarks_ids_right = MESH_ANNOTATIONS['rightEye']
landmarks_ids_left = MESH_ANNOTATIONS['leftEye']
scale = 1
filter_landmarks = False
lock_face = True

while True:
    ret, img = cap.read()

    if ret == False:
        break

    blank = np.ones(tuple([i * scale for i in img.shape[:2]])) * 255

    desiredLeftEye=(0.35, 0.35)
    desiredFaceWidth=img.shape[1]
    desiredFaceHeight=img.shape[0]
    if desiredFaceHeight is None:
        desiredFaceHeight = desiredFaceWidth

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)
    if results.multi_face_landmarks:
        for faceLms in results.multi_face_landmarks:
            if lock_face == False:
                landmarks = []
                if filter_landmarks:
                    for j, i in enumerate(landmarks_ids):
                        lm = faceLms.landmark[i]
                        h, w, _ = img.shape
                        x, y = int(lm.x * w) * scale, int(lm.y * h) * scale
                        cv2.circle(blank, (x, y), 1, (0, 0, 255), -1)
                        landmarks.append((x , y ))
                        # cv2.putText(blank, str(i), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                else:
                    for j, lm in enumerate(faceLms.landmark):
                        h, w, _ = img.shape
                        x, y = int(lm.x * w) * scale, int(lm.y * h) * scale
                        landmarks.append((x , y ))
                        cv2.circle(blank, (x, y), 1, (0, 0, 255), -1)
                        # cv2.putText(blank, str(j), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            else:
                landmarks_right = []
                for j, i in enumerate(landmarks_ids_right):
                    lm = faceLms.landmark[i]
                    h, w, _ = img.shape
                    x, y = int(lm.x * w), int(lm.y * h)
                    cv2.circle(blank, (x, y), 1, (0, 0, 255), -1)
                    landmarks_right.append((x, y))
                
                landmarks_left = []
                for j, i in enumerate(landmarks_ids_left):
                    lm = faceLms.landmark[i]
                    h, w, _ = img.shape
                    x, y = int(lm.x * w), int(lm.y * h)
                    cv2.circle(blank, (x, y), 1, (0, 0, 255), -1)
                    landmarks_left.append((x, y))

                # compute the center of mass for each eye
                leftEyeCenter = np.array(landmarks_left).mean(axis=0).astype("int")
                rightEyeCenter = np.array(landmarks_right).mean(axis=0).astype("int")
                # compute the angle between the eye centroids
                dY = rightEyeCenter[1] - leftEyeCenter[1]
                dX = rightEyeCenter[0] - leftEyeCenter[0]
                angle = np.degrees(np.arctan2(dY, dX)) - 180
                angle = 0

                # compute the desired right eye x-coordinate based on the
                # desired x-coordinate of the left eye
                desiredRightEyeX = 1.0 - desiredLeftEye[0]
                # determine the scale of the new resulting image by taking
                # the ratio of the distance between eyes in the *current*
                # image to the ratio of distance between eyes in the
                # *desired* image
                dist = np.sqrt((dX ** 2) + (dY ** 2))
                desiredDist = (desiredRightEyeX - desiredLeftEye[0])
                desiredDist *= desiredFaceWidth
                scale = desiredDist / dist
                scale = 1

                # compute center (x, y)-coordinates (i.e., the median point)
                # between the two eyes in the input image
                eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2,
                    (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
                # grab the rotation matrix for rotating and scaling the face
                M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
                # update the translation component of the matrix
                tX = desiredFaceWidth * 0.5
                tY = desiredFaceHeight * 0.2 # desiredLeftEye[1]
                M[0, 2] += (tX - eyesCenter[0])
                M[1, 2] += (tY - eyesCenter[1])

                # apply the affine transformation
                (w, h) = (desiredFaceWidth, desiredFaceHeight)
                img = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC)
                cv2.putText(blank, f'Tx: {tX} Ty: {tY}', (40, 150), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0), 2)
                cv2.putText(blank, f'eyesCenter[0]: {eyesCenter[0]} eyesCenter[1]: {eyesCenter[1]}', (40, 250), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0), 2)
                cv2.putText(blank, f'desiredFaceWidth: {desiredFaceWidth} desiredFaceHeight: {desiredFaceHeight}', (40, 350), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0), 2)

                # img = img[int(tY - eyesCenter[1]):, :] if int(tY - eyesCenter[1]) > 0 else img[:int(tY - eyesCenter[1]), :]
                # img = img[:, int(tX - eyesCenter[0]):] if int(tX - eyesCenter[0]) > 0 else img[:, :int(tX - eyesCenter[0])]

                img = img[:int(tY - eyesCenter[1]), :] if int(tY - eyesCenter[1]) < 0 else img
                img = img[:, :int(tX - eyesCenter[0])] if int(tX - eyesCenter[0]) < 0 else img

    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)

    if img.shape[0] > 0 and img.shape[1] > 0:
        cv2.imshow("webcam", img)
    cv2.imshow("mesh", blank)
    # k = cv2.waitKey(10)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

def magnify(array: np.ndarray):
    # media = array.mean()
    # idx_up_mean = np.where(array < media)
    # idx_down_mean = np.where(array > media)
    # array[idx_up_mean] = 255
    # array[idx_down_mean] = 0
    idx_max = np.where(array == max(array))
    idx_not_max = np.where(array != max(array))
    array[idx_max] = 255
    array[idx_not_max] = 0
    return array

cam = cv2.VideoCapture('/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still_940/RGB_20s/Frame%05d.pgm')
writer = cv2.VideoWriter('/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still_940/teste.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (640, 640))
video = np.zeros((30, 640, 640, 3))
cam_index = 0

while True:
    ret, frame = cam.read()

    if ret == False:
        print(f'O video acabou')
        break

    video[cam_index] = frame # [:,:,1]
    cam_index += 1

    if cam_index >= 30:
        print('Magnificando ...')
        cam_index = 0
        array = video.copy() # np.zeros((30, 640, 640, 3))
        for line in range(video.shape[1]):
            for col in range(video.shape[2]):
                if col > 150 and line > 50 and line < 550:
                    magnified = magnify(video[:, line, col, 1]) # , channel
                    for channel in range(video.shape[3]):
                        array[:, line, col, channel] = magnified
        video = np.zeros((30, 640, 640, 3))

        for frame in array:
            writer.write(frame.astype('uint8'))
        del array

cam.release()
writer.release()

In [ ]:
import cv2
import numpy as np

def show(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def scale_chn(img, chn, factor):
    img = img.astype('uint16')
    scale = int(2**(factor-1))
    if chn == 'r':
        b = img[:,:,0]
        g = img[:,:,1]
        r = np.clip(img[:,:,2]+scale-1, 0, 255)
    if chn == 'g':
        b = img[:,:,0]
        g = np.clip(img[:,:,1]+scale-1, 0, 255)
        r = img[:,:,2]
    if chn == 'b':
        b = np.clip(img[:,:,0]+scale-1, 0, 255)
        g = img[:,:,1]
        r = img[:,:,2]

    return cv2.merge((b.astype('uint8'), g.astype('uint8'), r.astype('uint8'))) 

bayer = np.right_shift(cv2.imread('/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still_940/teste/Frame00000_1.pgm',-1),8)

# Perform a Bayer Reconstruction
demosaic = cv2.cvtColor(bayer, cv2.COLOR_BAYER_BG2BGR)

# demosaic = scale_chn(demosaic, 'r', 5)
# demosaic = scale_chn(demosaic, 'r', 0)
# demosaic = scale_chn(demosaic, 'g', 0)
# demosaic = scale_chn(demosaic, 'g', 1)
# demosaic = scale_chn(demosaic, 'b', 1)

show(demosaic.astype('uint8'))
demosaic

In [1]:
import cv2
from tkinter import *
from PIL import Image, ImageTk

#Creating Tkinter Window and Label
root = Tk()
video = Label(root)
video.pack()

#Getting video from webcam
vid = cv2.VideoCapture("/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still_940/RGB_20s/Frame%05d.pgm")

#Loop which display video on the label
while(True):
    
    ret, frame = vid.read() #Reads the video

    if ret == False:
        print('O video terminou')
        break

    #Converting the video for Tkinter
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    img = Image.fromarray(cv2image)
    HEIGHT, WIDTH = img.size
    imgtk = ImageTk.PhotoImage(image=img, height=HEIGHT, width=WIDTH)

    #Setting the image on the label
    video.config(image=imgtk)

    root.update() #Updates the Tkinter window

vid.release()

cv2.destroyAllWindows()

# vlc_instance = vlc.Instance("--no-xlib")

# vlc_media_instance = vlc_instance.media_player_new()

# Media = vlc_instance.media_new(
#     '/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/MR-NIRP_Indoor/Subject1_still_940/RGB_20s/Frame%05d.pgm'
# )
# vlc_media_instance.set_media(Media)
# vlc_media_instance.play()
# vlc_media_instance.__dict__

O video terminou


In [1]:
import datetime
import logging
import numpy as np
import natsort
import cv2
import os
from utils import *

def demosaic_RGGB_video(video_path: str, out_path: str, video_name_pattern='Frame%05d.pgm'):
  """Funcao que recebe a estrutura de pasta para aplicar o demosaiced com o padrao RGGB em um video.

  - O padrao (video_name_pattern) so pode ter uma string com o seguinte formatador (%d ou nenhum):
    - (1) 'Frame%05d.pgm' no qual sera feito: pattern % 0

  Keyword arguments:
  - video_path: str -- ...
  - out_path: str -- Caminho da pasta na qual ...
  - video_name_pattern: str -- Padrao do nome da sequencia de imagens; ex.: 'Frame%05d.pgm' (%05d diz \
    que a sequencia vai ter cinco digitos, sendo que os primeiros vao ser preenchidos com zeros: Frame00000.pgm, \
      Frame00001.pgm, Frame00002.pgm, ...)

  Return: 
    None
  """
  assert os.path.isdir(out_path), "out_path deve ser um diretorio"

  start = video_name_pattern.split('%')[0]
  end = video_name_pattern.split('d')[-1]

  def frame_filter(filename: str):
    middle = filename.replace(start, '').replace(end, '')
    return filename.startswith(start) and filename.endswith(end) and middle.isdigit()

  if os.path.isdir(video_path) == False:
    # get parent folder
    video_dir = os.sep.join(video_path.split(os.sep)[:-1])
  else:
    video_dir = video_path

  frame_names = natsort.natsorted(filter(frame_filter, os.listdir(video_dir)))
  
  for frame_name in frame_names:
    frame_path = os.path.join(video_dir, frame_name)

    # based on: https://gist.github.com/bbattista/8358ccafecf927ae1c58c944ab470ffb
    frame = np.right_shift(cv2.imread(frame_path, cv2.IMREAD_UNCHANGED), 8)

    # Perform a Bayer Reconstruction
    frame = cv2.cvtColor(frame, cv2.COLOR_BAYER_BG2BGR).astype('uint8')

    # frame = cv2.cvtColor(frame.astype('uint8'), cv2.COLOR_BGR2GRAY)

    cv2.imwrite(os.path.join(out_path, frame_name.replace(end, '.png')), frame)

def demosaic_RGGB_dataset(dataset: Dataset, out_dataset_path: str, video_name_pattern='Frame%05d.pgm'):
  """Funcao que recebe a estrutura de pasta para aplicar o demosaiced em cada video com a funcao demosaic_video.

  - O padrao (video_name_pattern) so pode ter uma string com o seguinte formatador (%d ou nenhum):
    - (1) 'Frame%05d.pgm' no qual sera feito: pattern % 0

  - O objeto dataset deve pertencer a uma classe que herda da classe Dataset
  - A classe do objeto dataset deve ter implementado os metodos save_video e ppg_flatten

  - Todos os caminhos vao ser verificados durante o processo, um erro vai ser emitido sem prejudicar \
    os outros processos caso eles nao existam.

  Keyword arguments:
  - dataset: Dataset -- Objeto que pertence a uma classe que herda da classe Dataset e implementa todos os seus metodos.\
    dataset.paths: Iterable -- Lista de dicionarios que contem a seguinte estrutura:\n
    [
      {
        'subject_folder_name': 'Nome do sujeito ou da pasta do sujeito', 
        'subdir_folder_name': 'Nome da pasta interna que contem o video do sujeito', 
        'video_path': 'Caminho relativo ou absoluto do video com o nome do arquivo',
        'ppg_path': 'Caminho relativo ou absoluto do PPG em formato .mat com o nome do arquivo'
      },
      ...
    ]
  - out_dataset_path: str -- Caminho da pasta na qual deseja a estrutura final do dataset output (artificial).
  - video_name_pattern: str -- Padrao do nome da sequencia de imagens; ex.: 'Frame%05d.pgm' (%05d diz \
    que a sequencia vai ter cinco digitos, sendo que os primeiros vao ser preenchidos com zeros: Frame00000.pgm, \
      Frame00001.pgm, Frame00002.pgm, ...)

  Return: 
    None
  """

  assert issubclass(dataset.__class__, Dataset), "O objeto dataset deve pertencer a uma classe que herda da classe Dataset"

  exceptions_count = 0

  # para cada video que precisa ser processado
  for path in dataset.paths:
    subject_folder_name = path['subject_folder_name']
    subdir_folder_name = path['subdir_folder_name']
    video_path = path['video_path']

    out_path = os.path.join(out_dataset_path, subject_folder_name, 'RGB_demosaiced')
    make_dir(out_path)
    
    try:
      assert check_video_pattern(video_path, video_name_pattern, None), f"O seguinte video_path nao foi encontrado: {video_path}"
      demosaic_RGGB_video(
        video_path=video_path,
        out_path=out_path,
        video_name_pattern=video_name_pattern
      )
    except Exception as e:
      print_log(f"\nException nao mapeada: \n{logging.traceback.format_exc()}", None)
      exceptions_count += 1

  print_log(f"\nQuantidade de Exceptions: {exceptions_count}", None)
  print_log(f"\nFim da execucao: {datetime.datetime.now()}", None)

In [2]:
dataset = MRNirp()

dataset.map_dataset(
    base_dataset_path='/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste', 
    subdir_name=['RGB'], 
    video_name_pattern='Frame%05d.pgm'
)

demosaic_RGGB_dataset(
    dataset=dataset, 
    out_dataset_path='/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste'
)


Quantidade de Exceptions: 0

Fim da execucao: 2021-07-01 16:27:56.945851


In [1]:
from dataset_creator import process_dataset
from utils import MRNirp

dataset = MRNirp(image_extension='png')

dataset.map_dataset(
    base_dataset_path='/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste', 
    subdir_name=['RGB_demosaiced'], 
    video_name_pattern='Frame%05d.' + dataset.image_extension
)

process_dataset(
    dataset=dataset,
    out_dataset_path='/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out',
    video_name_pattern='Frame%05d.' + dataset.image_extension
)

Caminho do dataset artificial: /home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out
Padrao do nome dos videos: Frame%05d.png

Videos que vao ser processados:
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_1/RGB_demosaiced/Frame%05d.png
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_2/RGB_demosaiced/Frame%05d.png
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_3/RGB_demosaiced/Frame%05d.png
PPGs que vao ser processados:
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_1/PulseOX/pulseOx.mat
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_2/PulseOX/pulseOx.mat
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_3/PulseOX/pulseOx.mat

========== Video sujeito_1_RGB_demosaiced ==========

Processando video sujeito_1_RGB_demosaiced com os argumentos:
video_path: /home/desafio01/Documents/Codes/bio_hmd/D

Processing... |##                              | 1/12 - 1s

/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_1/RGB_demosaiced/bottom_face was created sucessfully
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_1/RGB_demosaiced/right_eye was created sucessfully
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_1/RGB_demosaiced/left_eye was created sucessfully


Processing... |################################| 12/12 - 0s



Quantidade de excecoes: 0

O video acabou
Processando PPG ...

========== Video sujeito_2_RGB_demosaiced ==========

Processando video sujeito_2_RGB_demosaiced com os argumentos:
video_path: /home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_2/RGB_demosaiced/Frame%05d.png
out_path: /home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_2/RGB_demosaiced
detector_type: FaceMesh
save_log: True
save_videos: True
realtime: False
use_threshold: True
distortion_fixed: False
use_frame_stabilizer: False
use_eyes_lock: False
use_video_stabilizer: False
threshold_modes: {'bottom_face': 'replace', 'right_eye': 'mean', 'left_eye': 'mean'}
dataset: <utils.MRNirp object at 0x7f400c636da0>
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_2/RGB_demosaiced was created sucessfully


...


Quantidade de excecoes: 0

O video acabou
Processando PPG ...

========== Video sujeito_3_RGB_demosaiced ==========

Processando video sujeito_3_RGB_demosaiced com os argumentos:
video_path: /home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_teste/sujeito_3/RGB_demosaiced/Frame%05d.png
out_path: /home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_3/RGB_demosaiced
detector_type: FaceMesh
save_log: True
save_videos: True
realtime: False
use_threshold: True
distortion_fixed: False
use_frame_stabilizer: False
use_eyes_lock: False
use_video_stabilizer: False
threshold_modes: {'bottom_face': 'replace', 'right_eye': 'mean', 'left_eye': 'mean'}
dataset: <utils.MRNirp object at 0x7f400c636da0>
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_3/RGB_demosaiced was created sucessfully


Processing... |##                              | 1/12 - 1s

/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_3/RGB_demosaiced/bottom_face was created sucessfully
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_3/RGB_demosaiced/right_eye was created sucessfully
/home/desafio01/Documents/Codes/bio_hmd/Dataset_MR_NIRP/dataset_out/sujeito_3/RGB_demosaiced/left_eye was created sucessfully


Processing... |################################| 12/12 - 0s


Quantidade de excecoes: 0

O video acabou
Processando PPG ...

Quantidade de Exceptions: 0

Quantidade de VideoExceptions: 0

Fim da execucao: 2021-07-02 21:01:58.258233
